In [2]:
import tensorflow as tf
import tensorflow.keras.datasets as tfds
from utils.layer_units import *
import pydotplus
import numpy as np

# Plot configurations
%matplotlib inline

# Notebook auto reloads code. (Ref: http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython)
%load_ext autoreload
%autoreload 2

In [14]:
# load cifar10 data
(X_train,y_train),(X_test,y_test) = tfds.cifar10.load_data()


In [15]:
X_valid = X_train[49000:]
y_valid = y_train[49000:]
X_train = X_train[:49000]
y_train = y_train[:49000]

In [16]:
# center and scale the input
def preprocess(X):
    # pixel wise center
    ret = X.astype(np.float32) - np.mean(X,axis=0)
    return ret / 255.0
    
    

In [17]:
X_train = preprocess(X_train)
X_valid = preprocess(X_valid)
X_test = preprocess(X_test)


In [4]:
ndist = tf.random.normal((1,64,64,3))
ipt = tf.keras.Input(shape=(32,32,3))
resid_layer = residual_units(ipt,[[64,3,1,0],[128,2,1,0],[66,3,1,0]])

In [7]:
resid_layer.get_shape

<bound method Tensor.get_shape of <tf.Tensor 'add/Identity:0' shape=(None, 32, 32, 66) dtype=float32>>

In [8]:
mock_model = tf.keras.Model(ipt,outputs=resid_layer)

In [9]:
mock_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 3)    12          input_1[0][0]                    
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 3)    0           batch_normalization[0][0]        
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 64)   1792        activation[0][0]                 
______________________________________________________________________________________________

In [10]:
ndist_pooled = tf.keras.layers.MaxPool2D(pool_size=(2,2))(ndist)
ndist_pooled.shape[-1]

3

In [11]:
[[2] for _ in range(3)]

[[2], [2], [2]]

In [40]:
hg_layer = hour_glass_unit(ipt,1,False)

In [41]:
mock_model = tf.keras.Model(ipt,outputs=hg_layer)
mock_model.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
max_pooling2d_18 (MaxPooling2D) (None, 16, 16, 3)    0           input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_213 (BatchN (None, 16, 16, 3)    12          max_pooling2d_18[0][0]           
__________________________________________________________________________________________________
activation_219 (Activation)     (None, 16, 16, 3)    0           batch_normalization_213[0][0]    
____________________________________________________________________________________________

In [37]:
dot_img_file = 'hour_glass_no_skip_connection.png'
tf.keras.utils.plot_model(mock_model,to_file=dot_img_file, show_shapes=True)

InvocationException: GraphViz's executables not found

In [26]:
ipt = tf.keras.Input(shape=(32,32,3))
attention_module1 = attention_unit(ipt,resid_layer_params = [[64,1,1,0],[128,3,1,0],[64,1,1,0]],skip_connection=True, skip_mode = 1)
resid1 = residual_units(attention_module1,[[64,1,1,0],[128,3,1,0],[64,1,1,0]])
attention_module2 = attention_unit(resid1,resid_layer_params = [[64,1,1,0],[128,3,1,0],[64,1,1,0]],skip_connection=True, skip_mode = 1)
resid2 = residual_units(attention_module2,[[64,1,1,0],[128,3,1,0],[64,1,1,0]])
attention_module3 = attention_unit(resid2,resid_layer_params = [[64,1,1,0],[128,3,1,0],[64,1,1,0]],skip_connection=True, skip_mode = 1)
resid3 = residual_units(attention_module3,[[64,1,1,0],[128,3,1,0],[64,1,1,0]])
resid4 = residual_units(resid3,[[64,1,1,0],[128,3,1,0],[64,1,1,0]])

In [25]:
mock_model = tf.keras.Model(ipt,outputs=resid3)
mock_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
batch_normalization_90 (BatchNo (None, 32, 32, 3)    12          input_2[0][0]                    
__________________________________________________________________________________________________
activation_93 (Activation)      (None, 32, 32, 3)    0           batch_normalization_90[0][0]     
__________________________________________________________________________________________________
conv2d_126 (Conv2D)             (None, 32, 32, 64)   256         activation_93[0][0]              
____________________________________________________________________________________________

In [25]:
dot_img_file = 'attention_unit.png'
tf.keras.utils.plot_model(mock_model,to_file=dot_img_file, show_shapes=True)

NameError: name 'mock_model' is not defined

In [27]:
# ImageDataGenerator code referred from hw2 Task4
# and referred from https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator
datagen = tf.keras.preprocessing.image.ImageDataGenerator(shear_range=0.1,zoom_range=0.2,horizontal_flip=True,rotation_range=30,width_shift_range=0.1,
                             height_shift_range=0.1)
datagen.fit(X_train)

In [28]:
avg0 = tf.keras.layers.GlobalAveragePooling2D()(resid4)
#flat1 = tf.keras.layers.Flatten()(avg0)
dense0 = tf.keras.layers.Dense(256,activation='relu')(avg0)
dense1 = tf.keras.layers.Dense(256,activation='relu')(dense0)
dense2 = tf.keras.layers.Dense(10,activation='softmax')(dense1)

model = tf.keras.Model(ipt,outputs=dense2)
opt = tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9,nesterov=True)

model.compile(optimizer = 'Adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['sparse_categorical_accuracy'],
)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
batch_normalization_93 (BatchNo (None, 32, 32, 3)    12          input_2[0][0]                    
__________________________________________________________________________________________________
activation_96 (Activation)      (None, 32, 32, 3)    0           batch_normalization_93[0][0]     
__________________________________________________________________________________________________
conv2d_130 (Conv2D)             (None, 32, 32, 64)   256         activation_96[0][0]              
____________________________________________________________________________________________

In [29]:
# tensorboards
# code referred from https://www.tensorflow.org/tensorboard/get_started
# clear logs
!rm -rf logs/

import datetime
log_dir = 'logs/fit/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1)

In [30]:
#history = model.fit(X_train,y_train,epochs=10,batch_size=128,validation_split=0.2,callbacks=[tensorboard_callback])
bs = 128
history = model.fit(datagen.flow(X_train,y_train,batch_size=bs),epochs=20,steps_per_epoch=len(X_train)//bs,
                    validation_data = (X_valid,y_valid))

Epoch 1/50
 27/382 [=>............................] - ETA: 4:10 - loss: 2.5446 - sparse_categorical_accuracy: 0.1817

KeyboardInterrupt: 

In [47]:
model.evaluate(x=X_test,y=y_test)

313/313 [==============================] - 18s 58ms/step - loss: 0.6642 - sparse_categorical_accuracy: 0.7928


[0.6642290949821472, 0.7928000092506409]

In [4]:
model.save('attention_with_skip_trainned_with_image_generator_80_percent.h5')

NameError: name 'model' is not defined

In [67]:
model.predict(np.expand_dims(X_test[19],axis=0))

array([[2.0254883e-12, 1.5670057e-10, 9.4586305e-08, 1.0920820e-07,
        4.8930165e-06, 3.7791081e-10, 9.9999487e-01, 8.8760067e-12,
        2.9913799e-10, 2.0425808e-09]], dtype=float32)

In [66]:
y_test[19]

array([6], dtype=uint8)

In [10]:
loaded_model = tf.keras.models.load_model('3_attention_layer_60_percent.h5')

In [11]:
loaded_model.evaluate(x=X_test,y=y_test)

313/313 [==============================] - 18s 58ms/step - loss: 1.2079 - sparse_categorical_accuracy: 0.5874


[1.2078672647476196, 0.5874000191688538]

In [12]:
# tensorboards
# code referred from https://www.tensorflow.org/tensorboard/get_started
# clear logs
!rm -rf logs/

import datetime
log_dir = 'logs/fit/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1)

In [13]:
history = loaded_model.fit(X_train,y_train,epochs=10,batch_size=128,validation_split=0.2,
          callbacks=[tensorboard_callback])

Epoch 1/10
313/313 [==============================] - 241s 770ms/step - loss: 0.8595 - sparse_categorical_accuracy: 0.6974 - val_loss: 1.2498 - val_sparse_categorical_accuracy: 0.5807
Epoch 2/10
313/313 [==============================] - 240s 768ms/step - loss: 0.7767 - sparse_categorical_accuracy: 0.7284 - val_loss: 1.0874 - val_sparse_categorical_accuracy: 0.6342
Epoch 3/10
313/313 [==============================] - 240s 767ms/step - loss: 0.6749 - sparse_categorical_accuracy: 0.7652 - val_loss: 1.0802 - val_sparse_categorical_accuracy: 0.6402
Epoch 4/10
313/313 [==============================] - 240s 767ms/step - loss: 0.5891 - sparse_categorical_accuracy: 0.7932 - val_loss: 1.2299 - val_sparse_categorical_accuracy: 0.6240
Epoch 5/10
313/313 [==============================] - 240s 768ms/step - loss: 0.5610 - sparse_categorical_accuracy: 0.8013 - val_loss: 1.3490 - val_sparse_categorical_accuracy: 0.6130
Epoch 6/10
313/313 [==============================] - 240s 768ms/step - loss: 0.

In [49]:
%load_ext tensorboard
# Load tensorboard on google cloud with specific port


In [69]:
%tensorboard --logdir=logs/fit/ --bind_all

Reusing TensorBoard on port 6006 (pid 4126), started 2:27:18 ago. (Use '!kill 4126' to kill it.)